In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os

import datetime
import plotly.graph_objects as go
from ipywidgets import widgets



In [2]:
pd.read_csv('../raw_data/LIGA/season-1415_csv.csv').loc[:, 'Div':'BbAH']

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH
0,SP1,23/08/14,Almeria,Espanol,1,1,D,0,0,D,...,3.35,3.18,2.90,2.75,44,2.34,2.23,1.70,1.64,26
1,SP1,23/08/14,Granada,La Coruna,2,1,H,0,1,A,...,3.45,3.23,4.40,3.99,36,2.61,2.43,1.61,1.54,24
2,SP1,23/08/14,Malaga,Ath Bilbao,1,0,H,1,0,H,...,3.40,3.23,2.80,2.55,45,2.14,2.02,1.85,1.77,26
3,SP1,23/08/14,Sevilla,Valencia,1,1,D,1,0,H,...,3.65,3.41,4.00,3.66,44,1.97,1.88,2.00,1.92,23
4,SP1,24/08/14,Barcelona,Elche,3,0,H,1,0,H,...,14.01,10.13,36.00,26.00,35,1.31,1.29,3.85,3.48,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,SP1,23/05/15,Granada,Ath Madrid,0,0,D,0,0,D,...,2.50,2.37,2.60,2.39,36,2.50,2.33,1.67,1.59,25
376,SP1,23/05/15,Levante,Elche,0,0,D,0,0,D,...,3.62,3.42,4.10,3.83,40,2.11,1.99,1.87,1.81,26
377,SP1,23/05/15,Malaga,Sevilla,2,3,A,0,0,D,...,4.00,3.73,2.21,2.13,37,1.60,1.48,2.70,2.60,25
378,SP1,23/05/15,Real Madrid,Getafe,7,3,H,3,3,D,...,12.00,9.49,29.00,20.36,29,1.22,1.18,5.40,4.51,22


In [3]:
league = ['BundesLiga','LIGA','Ligue1','PL','SerieA']

In [4]:
#directory_in_str = '/Users/antoinelemaire/code/Zorix9/beat_the_bookies/raw_data/LIGA'
directory_in_str = '../raw_data/LIGA'
directory = os.fsencode(directory_in_str)

list_df = [ pd.read_csv(directory_in_str+'/'+os.fsdecode(file)) for file in os.listdir(directory) if os.fsdecode(file).endswith('.csv')]
#print(f'{count} seasons in the dataframe')

In [5]:
list_df[0][('HomeTeam')].unique()

array(['Betis', 'Girona', 'Barcelona', 'Celta', 'Villarreal', 'Eibar',
       'Real Madrid', 'Vallecano', 'Ath Bilbao', 'Valencia', 'Getafe',
       'Leganes', 'Alaves', 'Ath Madrid', 'Valladolid', 'Espanol',
       'Sevilla', 'Levante', 'Huesca', 'Sociedad'], dtype=object)

In [6]:
for df in list_df:
    df['FTR_home'] = df['FTR'].apply(lambda x: 'Win' if str(x) == 'H' else ('Lose' if str(x) =='A' else 'Draw'))
    df['FTR_away'] = df['FTR'].apply(lambda x: 'Lose' if str(x) == 'H' else ('Win' if str(x)=='A' else 'Draw'))

In [7]:
list_df[0].head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,FTR_home,FTR_away
0,SP1,17/08/2018,Betis,Levante,0,3,A,0,1,A,...,-0.75,1.89,1.85,2.07,2.00,1.59,4.42,5.89,Lose,Win
1,SP1,17/08/2018,Girona,Valladolid,0,0,D,0,0,D,...,-0.75,2.06,2.01,1.90,1.85,1.76,3.57,5.62,Draw,Draw
2,SP1,18/08/2018,Barcelona,Alaves,3,0,H,0,0,D,...,-2.50,1.95,1.91,2.00,1.95,1.10,11.85,32.17,Win,Lose
3,SP1,18/08/2018,Celta,Espanol,1,1,D,0,1,A,...,-0.75,2.26,2.18,1.74,1.71,2.18,3.26,3.85,Draw,Draw
4,SP1,18/08/2018,Villarreal,Sociedad,1,2,A,1,1,D,...,-0.25,1.76,1.74,2.23,2.14,2.32,3.21,3.53,Lose,Win


# 1. Visualisation

In [8]:
team_stats = widgets.Dropdown(
    description='Teams:   ',
    value='Ath Madrid',
    options=list_df[4]['HomeTeam'].unique().tolist()
)


# Assign an empty figure widget with two traces
trace1 = go.Histogram(x=df['FTR_home'], opacity=0.75, name='Result at home over the year')
trace2 = go.Histogram(x=df['FTR_away'], opacity=0.75, name='Result away over the year')
g = go.FigureWidget(data=[trace1,trace2],
                    layout=go.Layout(
                        title=dict(
                            text='Team result'
                        ),
                        barmode='overlay'
                    ))

In [9]:
def validate():
    if team_stats.value in list_df[4]['HomeTeam'].unique():
        return True
    else:
        return False

def response(change):
    if validate():
        filter_list = [i for i in
                        df['HomeTeam'] == team_stats.value]
        temp_df = df[filter_list]
        x1 = temp_df['FTR_home']
        x2 = temp_df['FTR_away']
        with g.batch_update():
            g.data[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Result'
            g.layout.yaxis.title = 'Number of matchs'    

In [10]:
team_stats.observe(response, names='value')

In [11]:
container2 = widgets.HBox([team_stats])
widgets.VBox([container2,
              g])

# 1.1 Visualisation that works better


In [12]:
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,FTR_home,FTR_away
0,SP1,2009-08-29,Real Madrid,La Coruna,3,2,H,2,1,H,...,2.90,2.53,19,0.0,9.00,1.92,11.00,8.69,Win,Lose
1,SP1,2009-08-29,Zaragoza,Tenerife,1,0,H,0,0,D,...,1.91,1.79,18,0.0,1.38,1.36,3.20,3.01,Win,Lose
2,SP1,2009-08-30,Almeria,Valladolid,0,0,D,0,0,D,...,1.83,1.74,19,0.0,1.62,1.54,2.54,2.41,Draw,Draw
3,SP1,2009-08-30,Ath Bilbao,Espanol,1,0,H,0,0,D,...,1.81,1.74,19,0.0,1.66,1.58,2.47,2.30,Win,Lose
4,SP1,2009-08-30,Malaga,Ath Madrid,3,0,H,1,0,H,...,2.13,1.96,18,0.0,2.60,2.45,1.58,1.51,Win,Lose


In [13]:
list_season = ['09-10','10-11','11-12','12-13','13-14','14-15','15-16','16-17','17-18','18-19']

In [14]:
def bar_plt(season, team):
    ## season choice 
    dico = dict(zip(list_season[::-1],range(0,10)))
    df = list_df[dico[season]]
    ## for the home games
    try:
        filter_list = [i for i in
                        df['HomeTeam'] == team]
        temp_df = df[filter_list]
        x1 = temp_df['FTR_home']

        ## for the away games
        filter_list_away = [i for i in
                        df['AwayTeam'] == team]
        temp_df_away = df[filter_list_away]
        x2 = temp_df_away['FTR_away']

        x = ['Win','Draw','Lose']
        x_axis = np.arange(3)

        plt.bar(x_axis + 0.21,x1.value_counts()[['Win','Draw','Lose']],0.4,label='Home Games')
        plt.bar(x_axis - 0.21,x2.value_counts()[['Win','Draw','Lose']],0.4,label='Away Games')


        plt.xticks(x_axis, x)
        plt.xlabel("Result")
        plt.ylabel("Number of matches")
        plt.title(f"Result of {team} over the season {season}")
        plt.legend()
        plt.ylim([0,17])
    except:
        print(f'{team} wasnt playing during the year {season}')
    #plt.show()
    
    

In [16]:
widgets.interact(bar_plt,season=list_season,team=list(df['HomeTeam'].unique()));
#widgets.interact(bar_plt,season=list_season,team=['Ath Madrid','Real Madrid']);

interactive(children=(Dropdown(description='season', options=('09-10', '10-11', '11-12', '12-13', '13-14', '14…

# 1.2 Statistics of the teams

In [17]:
for df in list_df:
    df['Home_point'] = df['FTR'].apply(lambda x: 3 if str(x) == 'H' else (0 if str(x) =='A' else 1))
    df['Away_point'] = df['FTR'].apply(lambda x: 0 if str(x) == 'H' else (3 if str(x)=='A' else 1))

## 1.2.1 test classement

In [18]:
df1 = list_df[1]
df1.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,FTR_home,FTR_away,Home_point,Away_point
0,SP1,18/08/12,Celta,Malaga,0,1,A,0,0,D,...,1.96,1.98,1.91,2.35,3.45,3.29,Lose,Win,0,3
1,SP1,18/08/12,Mallorca,Espanol,2,1,H,1,1,D,...,1.98,1.95,1.90,2.25,3.33,3.63,Win,Lose,3,0
2,SP1,18/08/12,Sevilla,Getafe,2,1,H,2,0,H,...,1.82,2.12,2.05,1.68,3.96,5.73,Win,Lose,3,0
3,SP1,19/08/12,Ath Bilbao,Betis,3,5,A,0,3,A,...,2.12,1.81,1.76,2.17,3.49,3.65,Lose,Win,0,3
4,SP1,19/08/12,Barcelona,Sociedad,5,1,H,4,1,H,...,1.93,1.99,1.93,1.09,13.96,35.36,Win,Lose,3,0


In [19]:
lst_points = []
for item in df1['HomeTeam'].unique():
    lst_points.append(df1[df1['HomeTeam']==item]['Home_point'].sum() + df1[df1['AwayTeam']==item]['Away_point'].sum())
lst_points

[37,
 36,
 50,
 45,
 100,
 46,
 85,
 35,
 53,
 34,
 56,
 44,
 57,
 66,
 47,
 42,
 39,
 65,
 76,
 43]

In [20]:
classement = pd.DataFrame(list(zip(df1['HomeTeam'].unique(),lst_points)),columns=['team','points'])

In [21]:
classement = classement.sort_values(by='points',ascending=False).reset_index(drop=True)

## 1.2.2 Fonction classement 

In [22]:
def classment_per_year():
    '''
    Compute the classement of every season 
    ATTENTION : fct prend rien en paarmètre mais utilise la variable list_df
    '''
    list_df_classement = []
    for df in list_df:
        lst_points = []
        
        for item in df['HomeTeam'].unique():
           lst_points.append(df[df['HomeTeam']==item]['Home_point'].sum() + df[df['AwayTeam']==item]['Away_point'].sum())
        
        classement = pd.DataFrame(list(zip(df1['HomeTeam'].unique(),lst_points)),columns=['team','points'])
        classement = classement.sort_values(by='points',ascending=False).reset_index(drop=True)
        list_df_classement.append(classement)
    return list_df_classement

In [23]:
test_classement = classment_per_year()

In [24]:
test_classement[2]

,team,points
0,Granada,93
1,Sevilla,90
2,Levante,78
3,Barcelona,72
4,Valladolid,67
5,Real Madrid,64
6,Valencia,63
7,Espanol,56
8,Osasuna,55
9,Malaga,54


In [25]:
dic_season ={'09-10':9,'10-11':8,'11-12':7,'12-13':6,'13-14':5,'14-15':4,'15-16':3,'16-17':2,'17-18':1,'18-19':0}

In [26]:
list(dic_season.keys())

['09-10',
 '10-11',
 '11-12',
 '12-13',
 '13-14',
 '14-15',
 '15-16',
 '16-17',
 '17-18',
 '18-19']

## 1.2.3 pie faults


In [65]:
list_card = ['Red Card','Yellow Card','Fault']
new_df = []
new_df.append(list_df[0]['HR'].sum()+list_df[0]['AR'].sum())
new_df.append(list_df[0]['HY'].sum()+list_df[0]['AY'].sum())
new_df.append(list_df[0]['HF'].sum()+list_df[0]['AF'].sum() - new_df[0] - new_df[1])

df = pd.DataFrame(columns=list_card)

df.loc[0] = new_df
df

,Red Card,Yellow Card,Fault
0,79,1965,8247


In [66]:
import plotly.express as px
#df = px.data.tips()
fig = px.pie(df, values = df.loc[0], names=df.columns)
fig.show()

In [67]:
df.head()

,Red Card,Yellow Card,Fault
0,79,1965,8247


In [51]:
df[['size']].cumsum(axis=0)

,size
0,2
1,5
2,8
3,10
4,14
...,...
239,619
240,621
241,623
242,625


## 1.2.4 ranking top five

In [69]:
df2 = list_df[0]
df2.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,FTR_home,FTR_away,Home_point,Away_point
0,SP1,17/08/2018,Betis,Levante,0,3,A,0,1,A,...,1.85,2.07,2.00,1.59,4.42,5.89,Lose,Win,0,3
1,SP1,17/08/2018,Girona,Valladolid,0,0,D,0,0,D,...,2.01,1.90,1.85,1.76,3.57,5.62,Draw,Draw,1,1
2,SP1,18/08/2018,Barcelona,Alaves,3,0,H,0,0,D,...,1.91,2.00,1.95,1.10,11.85,32.17,Win,Lose,3,0
3,SP1,18/08/2018,Celta,Espanol,1,1,D,0,1,A,...,2.18,1.74,1.71,2.18,3.26,3.85,Draw,Draw,1,1
4,SP1,18/08/2018,Villarreal,Sociedad,1,2,A,1,1,D,...,1.74,2.23,2.14,2.32,3.21,3.53,Lose,Win,0,3


In [152]:
list_teams = ['Betis','Levante']
def filter_team(df2):
    ''' 
    Takes in input a dataframe of a season and gives back the results of "betis"
    To be adapted to all the teams and season
    '''
    filter_list = []
    point_list = []
    for index,item in enumerate(df2['HomeTeam']):
        if item == 'Betis':
            filter_list.append(True)
            if df2['FTR'][index] == 'H':
                point_list.append(3)
            elif df2['FTR'][index] == 'D':
                point_list.append(1)
            else:
                point_list.append(0)
        elif df2['AwayTeam'][index] == 'Betis':
            filter_list.append(True)
            if df2['FTR'][index] == 'A':
                point_list.append(3)
            elif df2['FTR'][index] == 'D':
                point_list.append(1)
            else:
                point_list.append(0)
        else:
            filter_list.append(False)
    temp_df = df2[filter_list]
    temp_df['points'] = point_list
    return temp_df
    #x1 = temp_df['FTR_home']
    #df['cumul_points'] = df['FTR'].apply(lambda x: 0 if str(x) == 'H' else (3 if str(x)=='A' else 1))

In [153]:
filter_team(list_df[0])

/var/folders/kp/t6tfj0414hvdyqqgmpmd5ytw0000gn/T/ipykernel_16816/2355289440.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,FTR_home,FTR_away,Home_point,Away_point,points
0,SP1,17/08/2018,Betis,Levante,0,3,A,0,1,A,...,2.07,2.00,1.59,4.42,5.89,Lose,Win,0,3,0
12,SP1,25/08/2018,Alaves,Betis,0,0,D,0,0,D,...,1.94,1.86,3.05,3.36,2.50,Draw,Draw,1,1,1
27,SP1,02/09/2018,Betis,Sevilla,1,0,H,0,0,D,...,2.19,2.13,2.84,3.60,2.53,Win,Lose,3,0,3
33,SP1,15/09/2018,Valencia,Betis,0,0,D,0,0,D,...,1.96,1.91,1.70,4.22,4.95,Draw,Draw,1,1,1
46,SP1,23/09/2018,Betis,Ath Bilbao,2,2,D,0,2,A,...,2.01,1.97,2.16,3.41,3.72,Draw,Draw,1,1,1
57,SP1,27/09/2018,Girona,Betis,0,1,A,0,0,D,...,1.76,1.73,2.54,3.38,2.98,Lose,Win,0,3,3
64,SP1,30/09/2018,Betis,Leganes,1,0,H,0,0,D,...,1.75,1.67,1.78,3.66,5.16,Win,Lose,3,0,3
74,SP1,07/10/2018,Ath Madrid,Betis,1,0,H,0,0,D,...,2.06,1.99,1.60,3.77,7.22,Win,Lose,3,0,0
84,SP1,21/10/2018,Betis,Valladolid,0,1,A,0,1,A,...,1.85,1.75,1.56,4.13,6.70,Lose,Win,0,3,0
98,SP1,28/10/2018,Getafe,Betis,2,0,H,0,0,D,...,1.88,1.83,2.65,3.10,3.08,Win,Lose,3,0,0


In [145]:
def compute_ranking(df2):
    df = filter_team(df2)
    list_points = []
    for index,item in enumerate(df['HomeTeam']):
        if df['FTR'][index] == 'D':
            list_points.append(1)
        elif item == 'Betis' and df['FTR'][index] == 'H':
            list_points.append(3)
        elif item == 'Betis' and df['FTR'][index] == 'A':
            list_points.append(0)
        elif df['FTR'][index] == 'H':
            list_points.append(0)
        else:
            list_points.append(3)
    df['points'] = list_points
    return filter_team(df2)

In [147]:
compute_ranking(list_df[0])

KeyError: 1

In [196]:
def test_func(lis):
    counter = 1
    while lis[counter] >= lis[counter-1]:
        if lis[counter] == lis[counter-1]:
            lis.append(lis[counter])
            lis.pop(counter)
        else:
            counter += 1
    return lis

In [197]:
l = [0,1,1,1,1,1,2,2,2,3,3,3,4,5,5]
test_func(l)


[0, 1, 2, 3, 4, 5, 1, 1, 1, 1, 2, 2, 3, 3, 5]

In [201]:
l = ['A', 'B', 'C']
l2 = [[1,2],[3,4],[5,6]]
pd.DataFrame(dict(zip(l,l2)))

,A,B,C
0,1,3,5
1,2,4,6
